Условие: Задача состоит в модификации кода градиентного спуска для задачи регрессии (ноутбук лекции - Gradient_descent.ipynb). Необходимо, чтобы ваш градиентный спуск мог решать задачи классификации, а именно линейной классификации и логистической регрессии.
На всякий случай напомню, что код будет разный, поэтому жду от вас две функции градиентного спуска.
Чтобы решить задачи, обратите внимание на пункт “Принцип максимального правдоподобия” статьи http://www.machinelearning.ru/wiki/images/6/68/voron-ML-Lin.pdf . Там находится ответ на вопрос “какая функция потерь”.

In [1]:
# first generate a data
import random
import pandas as pd
import numpy as np

# Don't run this cell

In [2]:
data = random.sample(range(1, 1000), 100)
data = pd.DataFrame(data)

# simple class determ function. odd/even
data['guess'] = data[0].apply(lambda x: x % 2)

# Don't run this cell

In [3]:
# the model crashed while trying to predict this
# was predictable, because it is not linear function
# let's do it another way

data = random.sample(range(1, 1000), 100)
data = pd.DataFrame(data)

def if_more(x):
    if x > 500:
        return 1
    else:
        return 0

# simple class determ function. odd/even
data['guess'] = data[0].apply(if_more)
data

,0,guess
0,602,1
1,566,1
2,725,1
3,380,0
4,607,1
...,...,...
95,636,1
96,23,0
97,459,0
98,367,0


In [4]:
# run only if you ran the one of previous two cells
x = data
x = x.to_numpy()
y = data['guess']
x[:5]

array([[602,   1],
       [566,   1],
       [725,   1],
       [380,   0],
       [607,   1]], dtype=int64)

# Linear classifier

In [5]:
def gradient_descent_class(alpha, x, y, numIterations):
    """
    Linear gradient classifier
    """
    
    m = x.shape[0] 
    
    # add ones for delta
    x = np.c_[ np.ones(m), x]
    
    n = x.shape[1]
    
    theta = np.ones(n) # model weights
    x_transpose = x.transpose()
    
    print(f"Initializing gradient search. M is {m}. N is {n}")
    
    for iter in range( 0, numIterations ):
        hypothesis = np.dot(x, theta) # матричное произведение
        
        loss = hypothesis - y
        J = np.sum(loss ** 2) / (2 * m)  # функция потерь
        
        #print( "iter %s | J: %.3f" % (iter, J) )
        #print(theta)
        
        gradient = np.dot(x_transpose, loss) / m         
        theta = theta - alpha * gradient  # update
    
    print('Model weights:')
    print(theta)
    return theta

In [6]:
# fuck it, take ready dataset
from sklearn import datasets
iris = datasets.load_iris()

prepare_data = pd.DataFrame(iris.data)
prepare_data['class'] = iris.target

# remove third class
prepare_data = prepare_data[prepare_data['class'] != 2] 

x = prepare_data.drop('class', axis=1)
x = x.to_numpy()
y = prepare_data['class']
y = y.to_numpy()

In [7]:
alpha = 0.01
theta = gradient_descent_class(alpha, x, y, 1000)

Initializing gradient search. M is 100. N is 5
Model weights:
[ 0.74630389 -0.25887133  0.04055461  0.17711531  0.68463828]


In [8]:
def class_prediction(row):
    global theta
    return theta[0] + theta[1]*row[0] + theta[2]*row[1] + theta[3]*row[2] + theta[4]*row[3]

results = pd.DataFrame(x)
results['class'] = y
results['predict'] = results.apply(class_prediction, axis=1)
results

,0,1,2,3,class,predict
0,5.1,3.5,1.4,0.2,0,-0.047110
1,4.9,3.0,1.4,0.2,0,-0.015613
2,4.7,3.2,1.3,0.2,0,0.026561
3,4.6,3.1,1.5,0.2,0,0.083816
4,5.0,3.6,1.4,0.2,0,-0.017167
...,...,...,...,...,...,...
95,5.7,3.0,4.2,1.2,1,0.957851
96,5.7,2.9,4.2,1.3,1,1.022260
97,6.2,2.9,4.3,1.3,1,0.910536
98,5.1,2.5,3.0,1.1,1,0.811895


In [9]:
from sklearn import linear_model
model = linear_model.SGDRegressor( alpha = 0.01, max_iter = 1000 )

model.fit( x, y )

SGDRegressor(alpha=0.01, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=1000,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [10]:
results['control'] = results.apply(lambda x: model.intercept_ + model.coef_[0]*x[0] 
                                   + model.coef_[1]*x[1] + model.coef_[2]*x[2] + model.coef_[3]*x[3], axis=1)

In [11]:
results

,0,1,2,3,class,predict,control
0,5.1,3.5,1.4,0.2,0,-0.047110,[-0.009035333990028177]
1,4.9,3.0,1.4,0.2,0,-0.015613,[0.05208301428620529]
2,4.7,3.2,1.3,0.2,0,0.026561,[-0.0016840380707527636]
3,4.6,3.1,1.5,0.2,0,0.083816,[0.06249858485815071]
4,5.0,3.6,1.4,0.2,0,-0.017167,[-0.022767497507691276]
...,...,...,...,...,...,...,...
95,5.7,3.0,4.2,1.2,1,0.957851,[0.9055211394723255]
96,5.7,2.9,4.2,1.3,1,1.022260,[0.9290099924992103]
97,6.2,2.9,4.3,1.3,1,0.910536,[0.9607001959009007]
98,5.1,2.5,3.0,1.1,1,0.811895,[0.6358626162996959]


# Logistic regression
http://neerc.ifmo.ru/wiki/index.php?title=%D0%9B%D0%BE%D0%B3%D0%B8%D1%81%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%81%D0%B8%D1%8F

In [49]:
import math


def activation_f(x,w):
    """
    Returns target function for logistic classifier
    x - parameters
    w - weights of model
    1 / (1 + e ** -x*w.transpose) Скалярное произведение
    """
    e_power = -1 * np.dot(x, w.transpose())
    return 1 / (1 + np.exp(e_power)) #


def gdc_log(alpha, x, y, numIterations):
    """
    Logistic gradient classifier
    """
    
    m = x.shape[0] 
    
    # add ones for delta
    x = np.c_[ np.ones(m), x]
    
    # model weights
    n = x.shape[1]
    w = np.ones(n)
    
    print(f"Initializing gradient search. M is {m}. N is {n}. X:")
    print(x)
    
    for iter in range( 0, numIterations ):
        # гипотеза
        a = activation_f(x,w) 
        
        # loss function
        loss = (-1) * y * np.log(a) - (1-y) * np.log(1 - a)
        loss = loss.sum()
        print(f"ITERATION {iter} | LOSS: {loss} | ACTIVATION: {a}  W:{w}")
        gradient = np.dot((a - y), x) 
        w = w - alpha * gradient  # update
    
    print('Model weights:')
    print(w)
    return w

In [13]:
alpha = 0.01
w = gdc_log(alpha, x, y, 1000)

Initializing gradient search. M is 100. N is 5. X:
[[1.  5.1 3.5 1.4 0.2]
 [1.  4.9 3.  1.4 0.2]
 [1.  4.7 3.2 1.3 0.2]
 [1.  4.6 3.1 1.5 0.2]
 [1.  5.  3.6 1.4 0.2]
 [1.  5.4 3.9 1.7 0.4]
 [1.  4.6 3.4 1.4 0.3]
 [1.  5.  3.4 1.5 0.2]
 [1.  4.4 2.9 1.4 0.2]
 [1.  4.9 3.1 1.5 0.1]
 [1.  5.4 3.7 1.5 0.2]
 [1.  4.8 3.4 1.6 0.2]
 [1.  4.8 3.  1.4 0.1]
 [1.  4.3 3.  1.1 0.1]
 [1.  5.8 4.  1.2 0.2]
 [1.  5.7 4.4 1.5 0.4]
 [1.  5.4 3.9 1.3 0.4]
 [1.  5.1 3.5 1.4 0.3]
 [1.  5.7 3.8 1.7 0.3]
 [1.  5.1 3.8 1.5 0.3]
 [1.  5.4 3.4 1.7 0.2]
 [1.  5.1 3.7 1.5 0.4]
 [1.  4.6 3.6 1.  0.2]
 [1.  5.1 3.3 1.7 0.5]
 [1.  4.8 3.4 1.9 0.2]
 [1.  5.  3.  1.6 0.2]
 [1.  5.  3.4 1.6 0.4]
 [1.  5.2 3.5 1.5 0.2]
 [1.  5.2 3.4 1.4 0.2]
 [1.  4.7 3.2 1.6 0.2]
 [1.  4.8 3.1 1.6 0.2]
 [1.  5.4 3.4 1.5 0.4]
 [1.  5.2 4.1 1.5 0.1]
 [1.  5.5 4.2 1.4 0.2]
 [1.  4.9 3.1 1.5 0.2]
 [1.  5.  3.2 1.2 0.2]
 [1.  5.5 3.5 1.3 0.2]
 [1.  4.9 3.6 1.4 0.1]
 [1.  4.4 3.  1.3 0.2]
 [1.  5.1 3.4 1.5 0.2]
 [1.  5.  3.5 1.3 0.3]
 [1.  

ITERATION 104 | LOSS: [4.23197893e-05 2.80450737e-04 1.13108027e-04 5.21404417e-04
 3.51011561e-05 6.90531696e-05 1.55644365e-04 1.14378812e-04
 7.67185117e-04 2.57515849e-04 2.50869266e-05 2.56394576e-04
 2.39178017e-04 9.53894972e-05 1.10835408e-06 3.12325739e-06
 8.15432479e-06 5.68860215e-05 4.71308110e-05 3.67560125e-05
 1.92729957e-04 6.82859499e-05 7.15817668e-06 9.74277304e-04
 1.27216915e-03 7.11781004e-04 3.52514134e-04 6.29753051e-05
 5.10229130e-05 5.61374538e-04 6.76785882e-04 1.19671608e-04
 6.72124559e-06 2.54364375e-06 3.46138531e-04 4.40142598e-05
 1.43652635e-05 2.99349306e-05 3.25461262e-04 9.97763960e-05
 3.82277154e-05 3.67053241e-03 1.70389885e-04 4.60828027e-04
 4.18330561e-04 4.32125421e-04 4.66458728e-05 2.21178603e-04
 2.87585908e-05 9.26707485e-05 7.63640959e-05 7.28456339e-05
 1.23216240e-05 3.02213991e-05 1.34154989e-05 1.38650638e-05
 2.24526945e-05 1.87755836e-03 3.84058500e-05 9.28973769e-05
 2.02859501e-04 9.56227740e-05 1.05136641e-04 8.46022172e-06
 2

 9.99938066e-01 9.99929633e-01 9.91152184e-01 9.99923168e-01]  W:[ 0.15762378 -1.33348542 -3.23508897  5.39080265  2.97923985]
ITERATION 222 | LOSS: [5.42131263e-05 3.56755979e-04 1.42265251e-04 6.60184217e-04
 4.48243086e-05 9.11051162e-05 1.96570299e-04 1.46767504e-04
 9.60044718e-04 3.28560123e-04 3.26218038e-05 3.28505848e-04
 3.02621374e-04 1.16973870e-04 1.43869376e-06 4.12147800e-06
 1.05445337e-05 7.30286893e-05 6.26511800e-05 4.74377896e-05
 2.53066339e-04 8.83102922e-05 8.84356588e-06 1.27474610e-03
 1.65445437e-03 9.17500276e-04 4.56543472e-04 8.13433512e-05
 6.55684441e-05 7.16759961e-04 8.66830207e-04 1.56233890e-04
 8.66820202e-06 3.30337846e-06 4.42571285e-04 5.56231098e-05
 1.85498367e-05 3.80217112e-05 4.05299534e-04 1.28447059e-04
 4.86715633e-05 4.59059110e-03 2.12232947e-04 5.99424893e-04
 5.52002310e-04 5.49075868e-04 6.03756356e-05 2.78684075e-04
 3.72749168e-05 1.18305649e-04 4.62952719e-05 4.53891250e-05
 7.40411525e-06 1.99847491e-05 8.29327210e-06 8.87907078e-

ITERATION 313 | LOSS: [5.43732891e-05 3.59820050e-04 1.42742685e-04 6.66539562e-04
 4.48981611e-05 9.19243646e-05 1.97501633e-04 1.47737494e-04
 9.68790395e-04 3.31436589e-04 3.27391827e-05 3.31449314e-04
 3.04797073e-04 1.16824794e-04 1.42838573e-06 4.11443158e-06
 1.05294729e-05 7.33366280e-05 6.32340001e-05 4.75938479e-05
 2.56198320e-04 8.88164398e-05 8.76206474e-06 1.29640163e-03
 1.68230603e-03 9.30313603e-04 4.61914742e-04 8.18063721e-05
 6.58479443e-05 7.24784214e-04 8.77676821e-04 1.57745042e-04
 8.64552384e-06 3.28776418e-06 4.47007493e-04 5.56869641e-05
 1.85661229e-05 3.80315065e-05 4.07454640e-04 1.29312883e-04
 4.87437336e-05 4.65936147e-03 2.12861657e-04 6.07291370e-04
 5.60299987e-04 5.54415655e-04 6.06553276e-05 2.80309195e-04
 3.74041470e-05 1.18918277e-04 4.14854623e-05 4.08657855e-05
 6.58534063e-06 1.81034355e-05 7.41159122e-06 7.98412224e-06
 1.24183772e-05 1.22471401e-03 2.13133795e-05 5.66973755e-05
 1.29287490e-04 5.56992224e-05 6.24917001e-05 4.72271107e-06
 1

ITERATION 430 | LOSS: [5.23720618e-05 3.49863571e-04 1.37996606e-04 6.49593471e-04
 4.31857234e-05 8.88846113e-05 1.91191108e-04 1.43024664e-04
 9.45320661e-04 3.22181200e-04 3.14843287e-05 3.22063525e-04
 2.96026125e-04 1.12654854e-04 1.35238418e-06 3.91621514e-06
 1.00607446e-05 7.07431028e-05 6.10927348e-05 4.58072662e-05
 2.49150920e-04 8.57597749e-05 8.34000626e-06 1.26998827e-03
 1.64935240e-03 9.09653292e-04 4.49857063e-04 7.90022092e-05
 6.35124286e-05 7.06976574e-04 8.57302013e-04 1.52951395e-04
 8.24990792e-06 3.12386023e-06 4.35175141e-04 5.36178258e-05
 1.78041647e-05 3.65393878e-05 3.95708705e-04 1.25143061e-04
 4.68968783e-05 4.58598137e-03 2.06005006e-04 5.92176225e-04
 5.46547142e-04 5.40074765e-04 5.84650864e-05 2.71894716e-04
 3.59833428e-05 1.14982871e-04 3.84062511e-05 3.79203304e-05
 6.03859884e-06 1.67811796e-05 6.81169932e-06 7.36073567e-06
 1.14536556e-05 1.16394809e-03 1.96882482e-05 5.29460442e-05
 1.21334204e-04 5.18740322e-05 5.81901497e-05 4.33539194e-06
 1

 9.99950739e-01 9.99944310e-01 9.92305445e-01 9.99938656e-01]  W:[ 0.14995936 -1.34064918 -3.28288088  5.47662047  3.02200937]
ITERATION 555 | LOSS: [4.98652706e-05 3.36579209e-04 1.31997453e-04 6.26581106e-04
 4.10625027e-05 8.48912790e-05 1.83115820e-04 1.36912032e-04
 9.13417339e-04 3.09827824e-04 2.99104423e-05 3.09536447e-04
 2.84495001e-04 1.07564700e-04 1.26362036e-06 3.67781857e-06
 9.49406529e-06 6.74595971e-05 5.82805032e-05 4.35677802e-05
 2.39459051e-04 8.18439549e-05 7.85103839e-06 1.23032713e-03
 1.59956557e-03 8.79948940e-04 4.33244284e-04 7.54091446e-05
 6.05550733e-05 6.82364555e-04 8.28589410e-04 1.46567128e-04
 7.77514868e-06 2.92985378e-06 4.19127412e-04 5.10563822e-05
 1.68673489e-05 3.47074606e-05 3.80494230e-04 1.19734669e-04
 4.46084696e-05 4.47243293e-03 1.97345688e-04 5.70980919e-04
 5.26910788e-04 5.20621536e-04 5.56891034e-05 2.60987542e-04
 3.42016715e-05 1.09943806e-04 3.60129486e-05 3.56098743e-05
 5.60656360e-06 1.57075267e-05 6.33322629e-06 6.85767741e-

ITERATION 674 | LOSS: [4.75846778e-05 3.24282081e-04 1.26511288e-04 6.05184192e-04
 3.91358829e-05 8.12261658e-05 1.75711545e-04 1.31299249e-04
 8.83703937e-04 2.98395857e-04 2.84810719e-05 2.97951456e-04
 2.73850471e-04 1.02939007e-04 1.18491485e-06 3.46468510e-06
 8.98495292e-06 6.44642788e-05 5.57017922e-05 4.15322848e-05
 2.30461100e-04 7.82640989e-05 7.41527371e-06 1.19286263e-03
 1.55245826e-03 8.52061618e-04 4.17792183e-04 7.21232875e-05
 5.78573762e-05 6.59417435e-04 8.01721172e-04 1.40679408e-04
 7.34952771e-06 2.75697166e-06 4.04227139e-04 4.87294444e-05
 1.60207938e-05 3.30488267e-05 3.66472235e-04 1.14770582e-04
 4.25314493e-05 4.36411952e-03 1.89418051e-04 5.51205545e-04
 5.08552627e-04 5.02545224e-04 5.31589748e-05 2.50949573e-04
 3.25829715e-05 1.05329011e-04 3.40340842e-05 3.36934051e-05
 5.25052113e-06 1.48124243e-05 5.93773617e-06 6.44019584e-06
 1.00575058e-05 1.07652187e-03 1.73417960e-05 4.74048681e-05
 1.09459606e-04 4.63291815e-05 5.18838557e-05 3.76717811e-06
 1

 9.99955475e-01 9.99949695e-01 9.92687843e-01 9.99944445e-01]  W:[ 0.14412191 -1.34996649 -3.31598502  5.52928564  3.04880977]
ITERATION 787 | LOSS: [4.55685283e-05 3.13295271e-04 1.21640252e-04 5.86017490e-04
 3.74351324e-05 7.79748280e-05 1.69129183e-04 1.26310142e-04
 8.57044062e-04 2.88185597e-04 2.72199645e-05 2.87611703e-04
 2.64351321e-04 9.88410507e-05 1.11658288e-06 3.27873982e-06
 8.53878487e-06 6.18122366e-05 5.34163451e-05 3.97347446e-05
 2.22425239e-04 7.50919057e-05 7.03448884e-06 1.15911623e-03
 1.50997985e-03 8.26987599e-04 4.03968267e-04 6.92106975e-05
 5.54689886e-05 6.38844365e-04 7.77592978e-04 1.35438522e-04
 6.97701068e-06 2.60634899e-06 3.90897552e-04 4.66724105e-05
 1.52759498e-05 3.15862204e-05 3.53957935e-04 1.10359998e-04
 4.06973083e-05 4.26586015e-03 1.82370305e-04 5.33491735e-04
 4.92105938e-04 4.86361556e-04 5.09211697e-05 2.42005107e-04
 3.11542069e-05 1.01231974e-04 3.23258008e-05 3.20363542e-05
 4.94547651e-06 1.40398735e-05 5.59842533e-06 6.08128151e-

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
# check results by standart libraries 
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs')
model = clf.fit(x, y)

In [15]:
def act(x,x_count,w):
    """
    Converts X and weights of model into prediction
    x_count - how much columns is X
    """
    
    x = x[:x_count]
    # add ones for delta
    x = np.concatenate([[1], x])
    
    return activation_f(x, w)

In [16]:
logr = pd.DataFrame(x)
logr['class'] = y

l = x.shape[1]
logr['predict'] = logr.apply(lambda x: act(x, l, w), axis=1)
logr['control'] = logr.apply(lambda x: act(x, l, np.concatenate([model.intercept_, model.coef_[0]])), axis=1)
logr

,0,1,2,3,class,predict,control
0,5.1,3.5,1.4,0.2,0,0.000042,0.016051
1,4.9,3.0,1.4,0.2,0,0.000294,0.022968
2,4.7,3.2,1.3,0.2,0,0.000113,0.014054
3,4.6,3.1,1.5,0.2,0,0.000553,0.023149
4,5.0,3.6,1.4,0.2,0,0.000035,0.014056
...,...,...,...,...,...,...,...
95,5.7,3.0,4.2,1.2,1,0.999922,0.982502
96,5.7,2.9,4.2,1.3,1,0.999959,0.985442
97,6.2,2.9,4.3,1.3,1,0.999954,0.990678
98,5.1,2.5,3.0,1.1,1,0.993001,0.794266


# Model scoring
## without using libs

In [31]:
def accurancy(y, p):
    tp = 0
    tn = 0
    fn = 0
    fp = 0
    
    for i in range(len(y)):
        if p[i]>0.5 and y[i] == 1:
            tp += 1
        elif p[i] > 0.5 and y[1] == 0:
            fp += 1
        elif p[i] < 0.5 and y[i] == 0:
            tn += 1
        else:
            fn += 1
    
    print(f'True positive: {tp} | True negative: {tn} | False positive: {fp} | False negative: {fn}')
    return (tp + tn) / (tp + tn + fp + fn)

acc = accurancy(y, logr['predict'])
print(f'My model accurancy {acc}')

acc = accurancy(y, logr['control'])
print(f'Standart score {acc}')            

True positive: 0 | True negative: 38896 | False positive: 0 | False negative: 34715
My model accurancy 0.5283992881498689
True positive: 25877 | True negative: 24426 | False positive: 14470 | False negative: 8838
Standart score 0.68336254092459


In [18]:
from sklearn.metrics import accuracy_score

def normalize(a):
    for i in range(len(a)):
        if a[i] > 0.5:
            a[i] = 1
        else:
            a[i] = 0
    
    return a
            
acc = accuracy_score(y, normalize(logr['predict']))
print(f'My model accurancy {acc}')

acc = accuracy_score(y, normalize(logr['control']))
print(f'Standart score {acc}')

My model accurancy 1.0
Standart score 1.0


D:\Python\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Python\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Perfect models... Need more data

In [19]:
taxi = pd.read_csv('taxi_log.csv')
taxi = taxi[taxi['distance_km'] >= 0]

In [20]:
X = taxi.drop('driver_response', axis=1).to_numpy()
y = taxi['driver_response'].to_numpy()
y

array([1, 0, 1, ..., 0, 0, 0], dtype=int64)

In [36]:
alpha = 0.01
w = gdc_log(alpha, X, y, 10)

clf = LogisticRegression(random_state=0, solver='lbfgs')
model = clf.fit(X, y)

Initializing gradient search. M is 73611. N is 6. X:
[[ 1.         55.805342   55.819329   18.802      25.217       3.42325973]
 [ 1.         55.813978   55.814827    6.747       9.8         0.41135558]
 [ 1.         55.803578   55.812559   12.383      19.25        1.06411596]
 ...
 [ 1.         55.961315   55.962203   46.983      46.217       0.20503258]
 [ 1.         55.961354   55.964892   50.029      51.1         0.39443479]
 [ 1.         55.961364   55.962322   54.811      56.183       0.43466928]]
ITERATION 0 | LOSS: nan | ACTIVATION: [1. 1. 1. ... 1. 1. 1.]  W:[1. 1. 1. 1. 1. 1.]
ITERATION 1 | LOSS: nan | ACTIVATION: [0. 0. 0. ... 0. 0. 0.]  W:[  -387.96       -21706.43002917 -21706.13715115 -12349.17551
 -14374.90921      -350.88923843]
ITERATION 2 | LOSS: nan | ACTIVATION: [0. 0. 0. ... 0. 0. 0.]  W:[  -40.81       -2353.04456785 -2352.94213245 -6055.1193
 -5366.62288       25.38706975]
ITERATION 3 | LOSS: nan | ACTIVATION: [1. 1. 1. ... 1. 1. 1.]  W:[  306.34       17000.3408

D:\Python\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in log
D:\Python\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in multiply


In [34]:
logr = pd.DataFrame(X)
l = X.shape[1]
w_model = np.concatenate([model.intercept_, model.coef_[0]])
logr['predict'] = logr.apply(lambda x: act(x, l, w), axis=1)

logr['control'] = logr.apply(lambda x: act(x, l, w_model), axis=1)
logr['class'] = y

acc = accurancy(y, logr['predict'].to_numpy())
print(f'My model accurancy {acc}.')
print(f'Weights {w}\n\n')


acc = accurancy(y, logr['control'])
print(f'Standart score {acc}')   
print(f'Weights {w_model}')

True positive: 0 | True negative: 38896 | False positive: 0 | False negative: 34715
My model accurancy 0.5283992881498689.
Weights [   -35.07        -2125.65167495  -2125.58013225 -18823.2915
 -15518.81394       266.98919021]


True positive: 25877 | True negative: 24426 | False positive: 14470 | False negative: 8838
Standart score 0.68336254092459
Weights [ 0.00147743  0.00303696  0.0025849  -0.05780542  0.02663625  0.12091457]


In [61]:
print(np.concatenate([[1],X[1]]))
print(w)
e_power = -1 * np.dot(np.concatenate([[1],X[1]]), w.transpose())
print(e_power)
1 / (1 + np.exp(e_power))

[ 1.         55.813978   55.814827    6.747       9.8         0.41135558]
[   -35.07        -2125.65167495  -2125.58013225 -18823.2915
 -15518.81394       266.98919021]
516290.3300459111


0.0

My function fails on this kind of dataset due to really huge power of epsilon in activation function